In [41]:
import requests
import json

In [42]:
userID = "kaneshige"
api_path = "https://kenkoooo.com/atcoder/atcoder-api/results?user="

In [43]:
# APIを用いた提出データの取得
def getSubmissionData(userID):
    api_url = api_path + userID
    response = requests.get(api_url)
    jsonData = response.json()
    return jsonData

In [44]:
submissions = getSubmissionData(userID)
submissions[:2]

[{'id': 25295913,
  'epoch_second': 1629716078,
  'problem_id': 'abc141_d',
  'contest_id': 'abc141',
  'user_id': 'kaneshige',
  'language': 'PyPy3 (7.3.0)',
  'point': 0.0,
  'length': 1016,
  'result': 'RE',
  'execution_time': 129},
 {'id': 25747481,
  'epoch_second': 1631361762,
  'problem_id': 'abc218_a',
  'contest_id': 'abc218',
  'user_id': 'kaneshige',
  'language': 'PyPy3 (7.3.0)',
  'point': 100.0,
  'length': 92,
  'result': 'AC',
  'execution_time': 61}]

In [45]:
# 各問題において最も新しいAC提出のみを取得する
# 各コンテストごとにまとめて返す
def collectNewestAcceptedSubmissions(submissions):
    sortedData = sorted(submissions, key=lambda x: x['id'])  # IDで昇順ソートすると古い順になる
    submits = {} # 各問題ごとに最新の提出に更新する
    for data in sortedData:
        if data["result"] != "AC": # ACだった提出だけ対象
            continue
        submits[data["problem_id"]] = data
    
    result = {} # コンテストごとにまとめる
    for sub in submits.values():
        if not sub["contest_id"] in result:
            result[sub["contest_id"]] = []
        result[sub["contest_id"]].append(sub)
    return result

In [46]:
newestSubmits =  collectNewestAcceptedSubmissions(submissions)
newestSubmits["abc230"][2]

{'id': 27659553,
 'epoch_second': 1638534801,
 'problem_id': 'abc230_b',
 'contest_id': 'abc230',
 'user_id': 'kaneshige',
 'language': 'PyPy3 (7.3.0)',
 'point': 200.0,
 'length': 715,
 'result': 'AC',
 'execution_time': 66}

In [47]:
import os

In [48]:
root = "submissions/"

for contestName in newestSubmits:
    path = root + contestName
    os.makedirs(path, exist_ok=True)

In [49]:
import re
import html
import chromedriver_binary
from selenium import webdriver
from time import sleep
import subprocess

In [50]:
driver = webdriver.Chrome()

# 追加したファイルの数を増やす
add_cnt = 0

for submissions in newestSubmits.values():
    for sub in submissions:
        # 問題番号の取得
        problem_num = sub["problem_id"][-1]
        
        # 古い問題の場合には数字になっているので、アルファベットに戻す
        if problem_num.isdigit():
            problem_num = chr(int(problem_num)+ord('a')-1)
        
        # 作成するファイルへのパス
        path = root + sub["contest_id"] + "/" + problem_num
        # 拡張子の設定（C++, Pythonのみ）
        if "C++" in sub["language"]:
            continue
            #path += ".cpp"
        elif "Python" in sub["language"] or "PyPy" in sub["language"]:
            path += ".py"
        
        # 既に提出コードがある場合は取得せず、次の問題の提出を探す
        if os.path.isfile(path): continue
        
        # 提出ページへアクセス
        sub_url = "https://atcoder.jp/contests/" + sub["contest_id"] + "/submissions/" + str(sub["id"])
        driver.get(sub_url)
        
        # 提出コードの取得
        code = driver.find_element_by_id("submission-code") 
        
        # code.text は提出時に含めていない空白が期待に反して含まれてしまう
        # 空白はシンタックスハイライティングによるものであるように見える
        # innerHTML から不要なタグなどを消し、空白が意図通りのテキストを得る
        inner_html = code.get_attribute('innerHTML')
        list_items = re.findall(r'<li[^>]*>.*?</li>', inner_html)
        lines = []
        for li in list_items:
            line1 = re.sub(r'<[^>]+>', '', li)
            line2 = re.sub(r'&nbsp;', '', line1)
            line3 = html.unescape(line2)
            lines.append(line3 + "\n")
        code_text = ''.join(lines)
            
        # 書き込み
        with open(path, 'w') as f:
            f.write(code_text)
        
        # C++の場合にはclang-formatを使ってフォーマットする
        #if "C++" in sub["language"]:
        #    subprocess.call(["clang-format", "-i",  "-style=file", path])
        
        # 追加したファイルの数を増やす
        add_cnt += 1
            
        # アクセス負荷軽減のために時間をおく(3秒)
        sleep(3)
        
driver.quit()

C:\Users\tange\AppData\Local\Temp/ipykernel_10164/3482802820.py:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  code = driver.find_element_by_id("submission-code")


In [51]:
print(add_cnt)

532


In [52]:
if add_cnt == 0:
    # 何も追加していなければGitにアクセスしない
    print("No added submissions, end process")
else:
    # GitHubにプッシュ
    import git
    import datetime

    dt_now = datetime.datetime.now()
    repo_url = "https://github.com/tishii2479/atcoder.git"
    repo = git.Repo()
    repo.git.add("submissions/*")
    repo.git.commit("submissions/*", message="add submission: " + dt_now.strftime('%Y/%m/%d %H:%M:%S'))
    repo.git.push("origin", "main")

    print(f"Finished process, added {add_cnt} files")

GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git push origin main
  stderr: 'error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/kaneshige922/AtCoder.git''